In [1]:
import pandas as pd
import numpy as np

In [7]:
# Load cleaned data
print("Loading data from telco_clean.csv...")
df = pd.read_csv('C:/churn_prediction/notebooks/data/processed/telco_clean.csv')

# 1. Create a feature for customer lifetime value
print("Creating 'CustomerLifetimeValue' feature...")
df['CustomerLifetimeValue'] = df['tenure'] * df['MonthlyCharges']

# 2. Create a feature for average monthly charges
print("Creating 'AvgMonthlyCharges' feature...")
df['AvgMonthlyCharges'] = df['TotalCharges'] / df['tenure']
# Handle division by zero
df['AvgMonthlyCharges'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['AvgMonthlyCharges'].fillna(df['MonthlyCharges'], inplace=True)

# 3. Create a feature for service count
print("Creating 'ServiceCount' feature...")
service_columns = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

# Initialize the count to 0
df['ServiceCount'] = 0

# Count the services that are active
for column in service_columns:
    # Consider a service active if it's not 'No' or 'No internet service' or 'No phone service'
    df['ServiceCount'] += np.where(~df[column].isin(['No', 'No internet service', 'No phone service']), 1, 0)

# 4. Create a binary feature for whether the customer has dependents or partners
print("Creating 'HasFamily' feature...")
df['HasFamily'] = np.where((df['Partner'] == 'Yes') | (df['Dependents'] == 'Yes'), 1, 0)

# 5. Create a binary feature for paperless billing and auto payment
print("Creating 'IsPaperlessBilling' and 'IsAutomaticPayment' features...")
df['IsPaperlessBilling'] = np.where(df['PaperlessBilling'] == 'Yes', 1, 0)
df['IsAutomaticPayment'] = np.where(df['PaymentMethod'].isin(['Bank transfer (automatic)', 'Credit card (automatic)']), 1, 0)

# 6. Create contract duration in months
print("Creating 'ContractDuration' feature...")
contract_mapping = {'Month-to-month': 1, 'One year': 12, 'Two year': 24}
df['ContractDuration'] = df['Contract'].map(contract_mapping)

# 7. Create a feature for the ratio of total charges to tenure
print("Creating 'ChargePerTenure' feature...")
df['ChargePerTenure'] = df['TotalCharges'] / df['tenure']
df['ChargePerTenure'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['ChargePerTenure'].fillna(df['MonthlyCharges'], inplace=True)

# 8. Create 'tenure_group' feature from the original code snippet
print("Creating 'tenure_group' feature...")
df['tenure_group'] = pd.qcut(df['tenure'], 4, labels=['0-25%', '25-50%', '50-75%', '75-100%'], duplicates='drop')

# 9. Create 'charge_ratio' feature from the original code snippet
print("Creating 'charge_ratio' feature...")
# Ensure TotalCharges is numeric, coerce errors, and fill NaNs
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)  # Fill NaNs
df['MonthlyCharges'].fillna(df['MonthlyCharges'].median(), inplace=True)  # Ensure no NaNs
df['tenure'].fillna(df['tenure'].median(), inplace=True)  # Ensure no NaNs

# Avoid division by zero or near-zero tenure/monthly charges
denominator = df['MonthlyCharges'] * df['tenure']
df['charge_ratio'] = np.where(denominator > 0.01, df['TotalCharges'] / denominator, 0)  # Set to 0 if denominator is too small

# Save the dataset with engineered features
df.to_csv('data/processed/telco_engineered.csv', index=False)

# Display new features
print("\nSummary of new features:")
new_features = ['CustomerLifetimeValue', 'AvgMonthlyCharges', 'ServiceCount',
                'HasFamily', 'IsPaperlessBilling', 'IsAutomaticPayment',
                'ContractDuration', 'ChargePerTenure', 'tenure_group', 'charge_ratio']
print(df[new_features].describe())

print("\nEngineering complete! Dataset saved as telco_engineered.csv")


# # Load cleaned data
# df = pd.read_csv('data/processed/telco_clean.csv')

# # 1. Create a feature for customer lifetime value
# df['CustomerLifetimeValue'] = df['tenure'] * df['MonthlyCharges']

# # 2. Create a feature for average monthly charges
# df['AvgMonthlyCharges'] = df['TotalCharges'] / df['tenure']
# # Handle division by zero
# df['AvgMonthlyCharges'].replace([np.inf, -np.inf], np.nan, inplace=True)
# df['AvgMonthlyCharges'].fillna(df['MonthlyCharges'], inplace=True)

# # 3. Create a feature for service count
# service_columns = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
#                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

# # Initialize the count to 0
# df['ServiceCount'] = 0

# # Count the services that are active
# for column in service_columns:
#     # Consider a service active if it's not 'No' or 'No internet service' or 'No phone service'
#     df['ServiceCount'] += np.where(~df[column].isin(['No', 'No internet service', 'No phone service']), 1, 0)

# # 4. Create a binary feature for whether the customer has dependents or partners
# df['HasFamily'] = np.where((df['Partner'] == 'Yes') | (df['Dependents'] == 'Yes'), 1, 0)

# # 5. Create a binary feature for paperless billing and auto payment
# df['IsPaperlessBilling'] = np.where(df['PaperlessBilling'] == 'Yes', 1, 0)
# df['IsAutomaticPayment'] = np.where(df['PaymentMethod'].isin(['Bank transfer (automatic)', 'Credit card (automatic)']), 1, 0)

# # 6. Create contract duration in months
# contract_mapping = {'Month-to-month': 1, 'One year': 12, 'Two year': 24}
# df['ContractDuration'] = df['Contract'].map(contract_mapping)

# # 7. Create a feature for the ratio of total charges to tenure
# df['ChargePerTenure'] = df['TotalCharges'] / df['tenure']
# df['ChargePerTenure'].replace([np.inf, -np.inf], np.nan, inplace=True)
# df['ChargePerTenure'].fillna(df['MonthlyCharges'], inplace=True)

# # Save the dataset with engineered features
# df.to_csv('data/processed/telco_engineered.csv', index=False)

# # Display new features
# new_features = ['CustomerLifetimeValue', 'AvgMonthlyCharges', 'ServiceCount', 
#                 'HasFamily', 'IsPaperlessBilling', 'IsAutomaticPayment', 
#                 'ContractDuration', 'ChargePerTenure']
# df[new_features].describe()

Loading data from telco_clean.csv...
Creating 'CustomerLifetimeValue' feature...
Creating 'AvgMonthlyCharges' feature...
Creating 'ServiceCount' feature...
Creating 'HasFamily' feature...
Creating 'IsPaperlessBilling' and 'IsAutomaticPayment' features...
Creating 'ContractDuration' feature...
Creating 'ChargePerTenure' feature...
Creating 'tenure_group' feature...
Creating 'charge_ratio' feature...


C:\Users\singh\AppData\Local\Temp\ipykernel_6984\1823221056.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AvgMonthlyCharges'].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\singh\AppData\Local\Temp\ipykernel_6984\1823221056.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh


Summary of new features:
       CustomerLifetimeValue  AvgMonthlyCharges  ServiceCount    HasFamily  \
count            7043.000000        7043.000000   7043.000000  7043.000000   
mean             2279.581350          64.762906      4.146244     0.534289   
std              2264.729447          30.189796      2.312720     0.498858   
min                 0.000000          13.775000      1.000000     0.000000   
25%               394.000000          35.935156      2.000000     0.000000   
50%              1393.600000          70.337500      4.000000     1.000000   
75%              3786.100000          90.174158      6.000000     1.000000   
max              8550.000000         121.400000      9.000000     1.000000   

       IsPaperlessBilling  IsAutomaticPayment  ContractDuration  \
count         7043.000000         7043.000000       7043.000000   
mean             0.592219            0.435326          8.835865   
std              0.491457            0.495835          9.551444   
min